<a href="https://colab.research.google.com/github/Naaaaaana/DWBI_Groupcw2/blob/main/CRM_Data_Mart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geotext
!pip install squarify
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import squarify
import geotext

#read csv file
df_Fact = pd.read_csv('/content/bank_transactions.csv')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00


In [ ]:
df_Fact.isna().sum()

,0
TransactionID,0
CustomerID,0
CustomerDOB,3397
CustGender,1100
CustLocation,151
CustAccountBalance,2369
TransactionDate,0
TransactionTime,0
TransactionAmount (INR),0


In [ ]:
df_Fact.shape

(1048567, 9)

In [ ]:
# connect to database
conn = sqlite3.connect("RFMdb")
cur = conn.cursor()

In [ ]:
# load CRM data into the RFMdb database
df_Fact.to_sql("data", conn)

1048567

In [ ]:
# CHECK IF DATA INSERTED INTO data/RFMdb
df_Fact1 = pd.read_sql('SELECT * FROM data', conn)
df_Fact1

,index,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...,...
1048562,1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


In [ ]:
# Replace empty strings with actual NaN
df_Fact1.replace('', pd.NA, inplace=True)

# Convert 'TransactionAmount (INR)' to numeric, turn invalids/blanks into NaN
# Using the actual column name 'TransactionAmount (INR)' instead of the variable Transactionamount
df_Fact1['TransactionAmount (INR)'] = pd.to_numeric(df_Fact1['TransactionAmount (INR)'], errors='coerce')

# Drop rows where transaction_amount is 0 or NaN
# Using the actual column name 'TransactionAmount (INR)' instead of the variable Transactionamount
df_Fact1 = df_Fact1[df_Fact1['TransactionAmount (INR)'].notna() & (df_Fact1['TransactionAmount (INR)'] != 0)]

# Drop rows where customer_dob is missing
df_Fact1 = df_Fact1[df_Fact1['CustomerDOB'].notna()]

# Drop rows where CustLocation is exactly "."
df_Fact1 = df_Fact1[df_Fact1['CustLocation'] != '.']

# Drop rows where ANY column has NaN
df_Fact1 = df_Fact1.dropna(how='any')

In [ ]:
print("After:", (df_Fact1['CustLocation'] == '.').sum())

After: 0


In [ ]:
df_Fact1.isna().sum()

,0
index,0
TransactionID,0
CustomerID,0
CustomerDOB,0
CustGender,0
CustLocation,0
CustAccountBalance,0
TransactionDate,0
TransactionTime,0
TransactionAmount (INR),0


In [ ]:
df_Fact1.shape

(1040708, 10)

In [ ]:
import re
from fuzzywuzzy import process

# Load the dataset
file_path = "/content/bank_transactions.csv"
df_Fact1 = pd.read_csv(file_path)

# Define a list of correct city names for reference (expand this list as needed)
city_list = ["MUMBAI", "DELHI", "BANGALORE", "HYDERABAD", "CHENNAI", "KOLKATA",
             "PUNE", "AHMEDABAD", "JAIPUR", "LUCKNOW", "SURAT", "KANPUR", "NAGPUR"]

# Function to extract potential city names from messy location text
def extract_city(text):
    if pd.isna(text) or text.strip() == ".":
        return None  # Handle NaN and "." values

    # Extract words (assuming cities are usually capitalized words in address format)
    words = re.findall(r'\b[A-Z][a-z]+\b', text)

    # Check if any word matches a known city
    for word in words:
        if word in city_list:
            return word

    return ""  # Return blank if no match is found

# Apply extraction function
df_Fact1["CustLocation"] = df_Fact1["CustLocation"].str.upper()  # Convert everything to uppercase
df_Fact1["CustLocationCleaned"] = df_Fact1["CustLocation"].apply(extract_city)

# Function to fix city name typos using fuzzy matching
def fix_city_name(city):
    if city is None:
        return None
    best_match = process.extractOne(city, city_list)
    return best_match[0] if best_match else city

# Apply typo correction
df_Fact1["CustLocationCleaned"] = df_Fact1["CustLocationCleaned"].apply(fix_city_name)

# Drop original CustLocation column (optional)
df_Fact1.drop(columns=["CustLocation"], inplace=True)

# Save cleaned file (optional)
df_Fact1.to_csv("/content/bank_transactions.csv", index=False)

# Show unique results
print(df_Fact1['CustLocationCleaned'].unique())  # Shows all unique cleaned city names